# mockup XY stage

simulate controller that takes (x,y) tuple as 2-D positioner

PV | meaning
:--- | :---
`mock:pair` | actual controller value: (x,y) position tuple
`mock:x` | virtual x positioner value
`mock:y` | virtual y positioner value

In [1]:
from ophyd import (PseudoPositioner, PseudoSingle, PVPositionerPC, EpicsSignal, EpicsSignalRO, Signal)
from ophyd import (Component as Cpt)
from ophyd.pseudopos import (pseudo_position_argument, real_position_argument)
from ophyd.status import Status

In [5]:
class SoftXYPositioner(PVPositionerPC):
    setpoint = Cpt(EpicsSignal, 'mock:pair.VAL')
    readback = Cpt(EpicsSignalRO, 'mock:pair.VAL')

xy_pos = SoftXYPositioner("", name="xy_pos")
xy_pos.wait_for_connection()
if len(xy_pos.position) == 0:
    # test data
    xy_pos.move((45.6789, 0.1234))
print(f"{xy_pos.position = }")
print(f"{xy_pos.done = }")
print(f"{xy_pos._done_moving() = }")
print(f"{xy_pos.moving = }")

xy_pos.position = array([ 0.1234, 45.6789])
xy_pos.done = None
xy_pos._done_moving() = None
xy_pos.moving = False


In [6]:
xy_pos.move(xy_pos.position[::-1])
print(f"{xy_pos.position = }")

xy_pos.position = array([45.6789,  0.1234])


In [2]:
# class MyXYStagePseudo(PseudoPositioner):
#     # The pseudo positioner axes:
#     x = Cpt(PseudoSingle)
#     y = Cpt(PseudoSingle)

#     # The real (or physical) positioners:
#     controller = Cpt(SoftXYPositioner, 'mock:pair')

#     @pseudo_position_argument
#     def forward(self, pseudo_pos):
#         '''Run a forward (pseudo -> real) calculation'''
#         print(f"forward: {pseudo_pos = }")
#         return self.RealPosition(x=pseudo_pos.controller[0],
#                                  y=pseudo_pos.controller[1])

#     @real_position_argument
#     def inverse(self, real_pos):
#         '''Run an inverse (real -> pseudo) calculation'''
#         print(f"inverse: {real_pos = }")
#         return self.PseudoPosition(controller=(real_pos.x, real_pos.y))

In [3]:
# xy_pos = SoftXYPositioner("mock:pair", name="xy_pos")
# xy_pos.move([5,6])
# print(f"{xy_pos.position = }")

In [4]:
# xy_stage = MyXYStagePseudo("", name="xy_stage")

In [12]:
xy_pos._setup_move??

Signature: xy_pos._setup_move(position)
Source:   
    def _setup_move(self, position):
        '''Move and do not wait until motion is complete (asynchronous)'''
        def done_moving(**kwargs):
            self.log.debug('%s async motion done', self.name)
            self._done_moving(success=True)

        if self.done is None:
            # No done signal, so we rely on put completion
            moving_val = 1 - self.done_value
            self._move_changed(value=moving_val)

        self.log.debug('%s.setpoint = %s', self.name, position)

        if self.actuate is not None:
            self.setpoint.put(position, wait=True)

            self.log.debug('%s.actuate = %s', self.name, self.actuate_value)
            self.actuate.put(self.actuate_value, wait=False,
                             callback=done_moving)
        else:
            self.setpoint.put(position, wait=False,
                              callback=done_moving)
File:      ~/.local/lib/python3.8/site-packages/op